In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
class GlobalAttention(nn.Module):
    def __init__(self, input_dim):
        super(GlobalAttention, self).__init__()
        
        self.input_dim = input_dim
        self.attention_weights = nn.Linear(input_dim, 1, bias=False)
        
    def forward(self, inputs):
        batch_size, seq_len, input_dim = inputs.size()
        attn_scores = self.attention_weights(inputs)
        attn_scores = attn_scores.view(batch_size, seq_len)
        attn_weights = F.softmax(attn_scores, dim=1)
        weighted_inputs = torch.bmm(attn_weights.unsqueeze(1), inputs).squeeze(1)
        return weighted_inputs, attn_weights

In [5]:
import torchtext

tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
vocab = torchtext.vocab.GloVe(name='6B', dim=100)

def prepare_input(sentence):
    tokens = tokenizer(sentence)
    vectorized_tokens = [vocab.stoi[token] for token in tokens]
    vectorized_tokens = torch.tensor(vectorized_tokens).unsqueeze(0)
    return vectorized_tokens

.vector_cache/glove.6B.zip: 862MB [08:52, 1.62MB/s]                               
100%|█████████▉| 399999/400000 [00:11<00:00, 34819.13it/s]


In [8]:
attn_layer = GlobalAttention(input_dim=100)
sentence = "This is a positive sentence."
vectorized_input = prepare_input(sentence)
weighted_inputs, attn_weights = attn_layer(vectorized_input)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x6 and 100x1)